# Challenge Scratchbook

* This notebook explores methods for the Kernel Methods for Machine Learning Kaggle [challenge](https://www.kaggle.com/c/kernel-methods-for-machine-learning-2018-2019/data).

* Note that this is a binary classification challenge.

Our first goal is to implement two baseline methods:
1. Random classification
2. All instances are 0s (Doing so we get an idea of the proportion of 0's in the public test set)
3. Implement the Simple Pattern Recognition Algorithm (SPR) from Learning with Kernels 

Before that, we have to implement some data loaders


Now that we are done with the above, our goal is to implement SVM with Gaussian kernel.

## Imports

In [1]:
import csv
import os
import numpy as np
from scipy import optimize
from tqdm import tqdm_notebook
import pandas as pd
from itertools import product

from utils.data import load_data, save_results
from utils.models import SVM, SPR
from utils.kernels import GaussianKernel, LinearKernel

## Paths and Globals

In [2]:
CWD = os.getcwd()
DATA_DIR = os.path.join(CWD, "data")
RESULT_DIR = os.path.join(CWD, "results")

FILES = {0: {"train_mat": "Xtr0_mat100.csv",
             "train": "Xtr0.csv",
             "test_mat": "Xte0_mat100.csv",
             "test": "Xte0.csv",
             "label": "Ytr0.csv"},
         1: {"train_mat": "Xtr1_mat100.csv",
             "train": "Xtr1.csv",
             "test_mat": "Xte1_mat100.csv",
             "test": "Xte1.csv",
             "label": "Ytr1.csv"},
         2: {"train_mat": "Xtr2_mat100.csv",
             "train": "Xtr2.csv",
             "test_mat": "Xte2_mat100.csv",
             "test": "Xte2.csv",
             "label": "Ytr2.csv"}}

## 0 entries

In [3]:
if False:
    with open(os.path.join(RESULT_DIR, "results.csv"), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(["Id", "Bound"])
        for i in range(3000):
            writer.writerow([i, 0])

**Comment:**

* We get 0.51266 which means that the dataset is pretty balanced.

## SPR

* Simple Pattern Recognition algorithm with Gaussian kernel

In [5]:
γ = 500
λ = 5e-5
kernel = GaussianKernel(γ)
#kernel = LinearKernel()

len_files = len(FILES)
for i in range(len_files):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    X_val = X_train[1600:]
    Y_val = Y_train[1600:]
    X_train = X_train[:1600]
    Y_train = Y_train[:1600]
    clf = SPR(kernel)
    clf.fit(X_train, Y_train)
    y_pred_train =clf.predict(X_train)
    y_pred_val = clf.predict(X_val)
    score_train = clf.score(y_pred_train, Y_train)
    score_val = clf.score(y_pred_val, Y_val)

    print(f"Accuracy on train set / val set {i} : {score_train} / {score_val} (λ: {λ},γ: {γ})")

Accuracy on train set / val set 0 : 0.99875 / 0.585 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 1 : 1.0 / 0.705 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 2 : 1.0 / 0.5775 (λ: 5e-05,γ: 500)


## Train and test on the different sets

In [4]:
results = np.zeros(3000)

for i in range(len(FILES)):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    clf = SPR()
    clf.fit(X_train, Y_train)
    results[i*1000:i*1000 + 1000] = clf.predict(X_test)

### Save results

In [6]:
# Test the save results function
save_results("test_results.csv", results, result_dir=RESULT_DIR)

## SVM with Gaussian Kernel

### Comparison with ``scikit-learn`` implementation

In [10]:
γ = 500
λ = 5e-5
kernel = GaussianKernel(γ)

len_files = len(FILES)
for i in range(len_files):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    X_val = X_train[1600:]
    Y_val = Y_train[1600:]
    X_train = X_train[:1600]
    Y_train = Y_train[:1600]
    clf = SVM(_lambda=λ, kernel=kernel)
    clf.fit(X_train, Y_train)
    y_pred_train =clf.predict(X_train)
    y_pred_val = clf.predict(X_val)
    score_train = clf.score(y_pred_train, Y_train)
    score_val = clf.score(y_pred_val, Y_val)

    print(f"Accuracy on train set / val set {i} : {score_train} / {score_val} (λ: {λ},γ: {γ})")

Accuracy on train set / val set 0 : 1.0 / 0.575 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 1 : 1.0 / 0.7275 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 2 : 1.0 / 0.6375 (λ: 5e-05,γ: 500)


In [11]:
n = 2000
print(f"C: {1/(2 * n * λ)}")

C: 5.0


In [13]:
from sklearn.svm import SVC

len_files = len(FILES)
for i in range(len_files):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    X_val = X_train[1600:]
    Y_val = Y_train[1600:]
    X_train = X_train[:1600]
    Y_train = Y_train[:1600]
    clf = SVC(C=5.0, kernel="rbf", gamma=500)
    clf.fit(X_train, Y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_val = clf.predict(X_val)
    score_train = np.sum([Y_train == y_pred_train]) / len(Y_train)
    score_val = np.sum([Y_val == y_pred_val]) / len(Y_val)

    print(f"Accuracy on train set / val set {i} : {score_train} / {score_val} (λ: {λ},γ: {γ})")

Accuracy on train set / val set 0 : 1.0 / 0.5725 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 1 : 1.0 / 0.705 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 2 : 1.0 / 0.5875 (λ: 5e-05,γ: 500)


### Investigate and improve Kmeans

* Diagnostic: introduce a tolerance parameter ``tol``

### Cross-validation

In [69]:
def cross_validation(dataset_idx, clf, k=5):
    """
    Perform a k-fold cross-validation on a specific dataset
    given a specific classifier
    
    
    Parameters
    -------------
    - dataset_idx : int
        Dataset index to be called in the data loader
        
    - clf : object
        Classifier object with methods:
        . fit
        . predict
        . score
        
    - k : int
        Number of folds
    
    Returns
    -----------
    - results : dictionary
        Summary of the results
        Note: the results can be display using
        a pandas.DataFrame such as in:
        ``pd.DataFrame(results)``
    """
    
    # Setup
    scores_val = list()
    scores_train = list()
    
    # Load data
    X_train, Y_train, X_test = load_data(dataset_idx, data_dir=DATA_DIR, files_dict=FILES)
    
    n = len(X_train)
    assert n == len(Y_train)
    
    # Divise the samples
    bounds = [(i * (len(X_train) // k), (i+1) * (len(X_train) // k))
              for i in range(k)]


    # Loop through the divided samples
    for bound in bounds:
        lower, upper = bound
        # Create index array for validation set
        idx = np.arange(lower, upper)
        not_idx = [i for i in range(n) if i not in idx]

        # Populate current train and val sets
        _X_val = X_train[idx]
        _Y_val = Y_train[idx]
        _X_train = X_train[not_idx]
        _Y_train = Y_train[not_idx]

        # Sanity checks
        assert len(_X_train) == len(_Y_train)
        assert len(_X_val) == len(_Y_val)
        assert len(_X_train) == n - len(X_train) // k

        # Fit the classifier on the current training set
        clf.fit(_X_train, _Y_train)
        # Compute the score
        y_pred_train = clf.predict(_X_train)
        y_pred_val = clf.predict(_X_val)
        score_train = clf.score(y_pred_train, _Y_train)
        score_val = clf.score(y_pred_val, _Y_val)

        scores_val.append(score_val)
        scores_train.append(score_train)


   
    # Format the results in a dictionary
    # Compute the score average and standard deviation
    results = {"train_scores": scores_train,
               "val_scores": scores_val,
               "train_avg": np.mean(scores_train),
               "val_avg": np.mean(scores_val),
               "train_std": np.std(scores_train),
               "val_std": np.std(scores_val)}
    
    return results

In [68]:
clf = SVM(_lambda=λ, kernel=kernel)

cross_validation(0, clf)

{'train_scores': [1.0, 1.0, 1.0, 1.0, 1.0],
 'val_scores': [0.5625, 0.5675, 0.575, 0.6, 0.575],
 'train_avg': 1.0,
 'val_avg': 0.576,
 'train_std': 0.0,
 'val_std': 0.012903487900563932}

### Example of Grid Search using Cross Validation

In [73]:
# Setup
γ = 350
λ = 1e-5
gamma_list = np.linspace(50,γ,5, endpoint = True)
lambda_list = np.linspace(5e-7, λ, 5, endpoint = True)

settings = list(product(gamma_list,lambda_list))

best_score = {i: 0 for i in range(3)}
best_lambda = {i: 0 for i in range(3)}
best_gamma = {i: 0 for i in range(3)}

for k, tup in enumerate(settings):
    
    γ, λ = tup
    kernel = GaussianKernel(γ)
    
    len_files = len(FILES)
    clf = SVM(_lambda=λ, kernel=kernel)
    
    for i in range(len_files):
        # cross validation (default: k=5)
        results = cross_validation(i, clf)
        score_train = results["train_avg"]
        score_val = results["val_avg"]
        print(f"Accuracy on train set / val set {i} : {round(score_train, 3)} / {round(score_val, 3)} (λ: {λ},γ: {γ})")
        
        if score_val > best_score[i]:
            best_score[i] = score_val
            best_lambda[i] = λ
            best_gamma[i] = γ
        
    print('\n')

Accuracy on train set / val set 0 : 1.0 / 0.55 (λ: 5e-07,γ: 50.0)
Accuracy on train set / val set 1 : 1.0 / 0.68 (λ: 5e-07,γ: 50.0)
Accuracy on train set / val set 2 : 1.0 / 0.615 (λ: 5e-07,γ: 50.0)


Accuracy on train set / val set 0 : 1.0 / 0.55 (λ: 2.875e-06,γ: 50.0)
Accuracy on train set / val set 1 : 1.0 / 0.68 (λ: 2.875e-06,γ: 50.0)
Accuracy on train set / val set 2 : 1.0 / 0.615 (λ: 2.875e-06,γ: 50.0)


Accuracy on train set / val set 0 : 1.0 / 0.551 (λ: 5.2500000000000006e-06,γ: 50.0)
Accuracy on train set / val set 1 : 1.0 / 0.68 (λ: 5.2500000000000006e-06,γ: 50.0)
Accuracy on train set / val set 2 : 1.0 / 0.615 (λ: 5.2500000000000006e-06,γ: 50.0)


Accuracy on train set / val set 0 : 1.0 / 0.551 (λ: 7.625000000000001e-06,γ: 50.0)


KeyboardInterrupt: 

## Zero-padding implementation

In [86]:
ENCODING = {'A': [1.,0.,0.,0.],
            'C': [0.,1.,0.,0.],
            'G': [0.,0.,1.,0.],
            'T': [0.,0.,0.,1.],
            'Z': [0.,0.,0.,0.]} # used in zero-padding

def P(i, seq, k, zero_padding=True):
    """
    Compute the a k_mers at a given position in a nucleotides sequence
    
    Parameters
    -----------
    - i : int
        Position in the sequence
        
    - k : int
        Size of k-mer to be returned
        
    - seq : str
        Sequence of nucleotides
        
    - zero_padding : boolean (optional)
        Whether to use zero-padding on the sequence edges
        Default: True
        
    Returns
    -----------
    - L : numpy.array
        One-hot encoding of the string sequence
        
    - not_in : boolean
        Whether the k-mer was computed on the sequence edges
        Always set to False when using zero-padding
    """
    # Setup
    not_in = True
    if zero_padding:
        not_in = False
    
    # lower edge
    if i-(k+1)//2 + 1 < 0:
        # Use heading zero padding here
        n_zeros = abs(i - (k+1) // 2 + 1)
        k_mer_i = 'Z'*n_zeros + seq[:  i + (k+2)//2]
    # upper edge
    elif i + (k+2)//2 > len(seq):
        # Use trailing zero padding here
        n_zeros = i + (k+2) // 2 - len(seq)
        k_mer_i = seq[i - (k+1)//2 + 1:] + 'Z'*n_zeros
    # in the middle
    else:
        k_mer_i = seq[i-(k+1)//2 + 1 :  i + (k+2)//2]
        not_in = False
        
    # concatenate one hot encoding
    L = []
    for c in k_mer_i:
        L += ENCODING[c]
    
    # Sanity check
    assert len(L) == 4 * k
    
    # Convert to array and return
    return np.array(L), not_in

## TODO: CKN implementation